In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
import ezodf

def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
#leitura da tabela
df = read_ods(filename='ensino_medio_full_semCPF_semNome_grad.ods')

In [2]:
class Disciplina():
    def __init__(self,codigo,area_ensino,quantidade_aulas_semanais,numero_turma,inicio_aula,fim_aula,professor,numero_maximo_diario,numero_maximo_sequencia,index):
        self.codigo=codigo
        self.area_ensino=area_ensino
        self.quantidade_aulas_semanais=quantidade_aulas_semanais
        self.numero_turma=numero_turma
        self.inicio_aula=inicio_aula
        self.fim_aula=fim_aula
        self.professor=professor
        self.numero_maximo_diario=numero_maximo_diario
        self.numero_maximo_sequencia=numero_maximo_sequencia
        self.index=index
        
    def nome(self):
        switcher = {
            1344: "2º PROFESSOR - 20",
            628: "ARTE",
            612: "CIÊNCIAS",
            611: "RELIGIÃO",
            536: "FILOSOFIA",
            513: "QUÍMICA",
            475: "FÍSICA",
            437: "SOCIOLOGIA",
            401: "LITERATURA",
            319: "INGLÊS",
            307: "EDUCAÇÃO FÍSICA",
            304: "HISTÓRIA",
            302: "GEOGRAFIA",
            301: "MATEMÁTICA",
            255: "BIOLOGIA",
            202: "PORTUGUES",
            1: "LETRAS - ENS. FUND.- ANOS INICIAIS"
        }
 
        # get() method of dictionary data type  returns
        # value of passed argument if it is present
        # in dictionary otherwise second argument will
        # be assigned as default value of passed argument
        return switcher.get(self.codigo, "code not found")
    
    def nome_area_ensino(self):
        switcher = {
            1: "ENS.FUND.ANOS INICIAIS",
            2: "ENS.FUND.ANOS FINAIS",
            3: "ENSINO MÉDIO",
        }
 
        # get() method of dictionary data type  returns
        # value of passed argument if it is present
        # in dictionary otherwise second argument will
        # be assigned as default value of passed argument
        return switcher.get(self.area_ensino, "code not found")
    
    def get_codigo(self):
        return self.codigo
    
    def get_area_ensino(self):
        return self.area_ensino
    
    def get_quantidade_aulas_semanais(self):
        return self.quantidade_aulas_semanais
    
    def get_numero_turma(self):
        return self.numero_turma

    def get_inicio_aula(self):
        return self.inicio_aula
    
    def get_fim_aula(self):
        return self.fim_aula
    
    def get_professor(self):
        return self.professor

    def get_numero_maximo_diario(self):
        return self.numero_maximo_diario
    
    def get_numero_maximo_sequenciar(self):
        return self.numero_maximo_sequencia
    
    def get_index(self):
        return self.index
    
    def get_horarios(self):
        horarios=[]
        if (self.inicio_aula == "07:45" and self.fim_aula=="11:45"):
            for x in range (2,7):
                for y in range (1,6):
                    horarios.append([x,y])
        if (self.inicio_aula == "13:30" and self.fim_aula=="17:30"):
            horarios=[]
            for x in range (2,7):
                for y in range (6,11):
                    horarios.append([x,y])
        return horarios
                    

In [3]:
class Turma():
    def __init__(self,codigo,area_ensino,serie,index):
        self.codigo=codigo
        self.area_ensino=area_ensino
        self.serie=serie
        self.index=index
            
    def nome_area_ensino(self):
        switcher = {
            1: "ENS.FUND.ANOS INICIAIS",
            2: "ENS.FUND.ANOS FINAIS",
            3: "ENSINO MÉDIO",
        }
 
        # get() method of dictionary data type  returns
        # value of passed argument if it is present
        # in dictionary otherwise second argument will
        # be assigned as default value of passed argument
        return switcher.get(self.area_ensino, "code not found")
    
    def get_codigo(self):
        return self.codigo
    
    def get_area_ensino(self):
        return self.area_ensino
    
    def get_serie(self):
        return self.serie
    
    def get_index(self):
        return self.index

In [4]:
class Professor():
    def __init__(self,codigo,nome,disciplinas,index):
        self.codigo=codigo
        self.nome=nome
        self.disciplinas=disciplinas
        self.index=index

    def get_codigo(self):
        return self.codigo
    
    def get_nome(self):
        return self.nome
    
    def get_carga_horaria(self):
        carga_horaria=0
        for x in range(len(self.disciplinas)):
            indice = self.disciplinas[x]
            disciplina=objs[indice]
            carga_horaria_temp = disciplina.get_quantidade_aulas_semanais()
            carga_horaria = carga_horaria + carga_horaria_temp
        return carga_horaria
    
    def get_disciplinas(self):
        return self.disciplinas

    def get_index(self):
        return self.index
    
    def get_horarios(self):
        horarios=[]
        
        for x in range(len(self.disciplinas)):
            indice = self.disciplinas[x]
            disciplina=objs[indice]
            
            if (disciplina.get_inicio_aula() == "07:45" and disciplina.get_fim_aula()=="11:45"):  
                for x in range (2,7):
                    for y in range (1,6):
                        horarios.append([x,y])
            if (disciplina.get_inicio_aula() == "13:30" and disciplina.get_fim_aula()=="17:30"):
                for x in range (2,7):
                    for y in range (6,11):
                        horarios.append([x,y])
       
        temp_list = []
        for i in horarios :
            if i not in temp_list:
                temp_list.append(i)
        horarios = temp_list

        return horarios

In [5]:
class Grupo():
    def __init__(self,codigo,nome,maximo,disciplinas,index):
        self.codigo=codigo
        self.nome=nome
        self.maximo=maximo
        self.disciplinas=disciplinas
        self.index=index

    def get_codigo(self):
        return self.codigo
    
    def get_nome(self):
        return self.nome
    
    def get_maximo(self):
        return self.maximo
    
    def get_disciplinas(self):
        return self.disciplinas

    def get_index(self):
        return self.index

In [6]:
class Horario():
    def __init__(self,codigo,dia_semana,inicio,turno,index):
        self.codigo=codigo
        self.dia_semana=dia_semana
        self.inicio=inicio
        self.turno=turno
        self.index=index
        
    def get_codigo(self):
        return self.codigo

    def get_dia_semana(self):
        return self.dia_semana
    
    def get_inicio(self):
        return self.inicio
    
    def get_turno(self):
        return self.turno
    
    def get_index(self):
        return self.index

In [7]:
#retorna uma lista com objetos disciplinas que sao lecionadas para a turma
def disciplinas_turma(turma,disciplinas):    
    retorno=[]
    for x in range(len(disciplinas)):
            disciplina=disciplinas[x]
            if (disciplina.get_numero_turma() == turma.get_codigo()):  
                        retorno.append(disciplina)

    return retorno

In [8]:
#listar todas as turmas(objetos) de um professor
def listar_turmas_professor(professor,objs_turmas,objs):
    retorno = []
    disciplinas_professor=professor.get_disciplinas()
    for x in range(len(disciplinas_professor)):
        indice=disciplinas_professor[x]
        disciplina=objs[indice]
        for y in range(len(objs_turmas)):
            turma=objs_turmas[y]
            if (disciplina.get_numero_turma() == turma.get_codigo()):
                retorno.append(turma)
    
    temp_list = []
    for i in retorno :
        if i not in temp_list:
            temp_list.append(i)

    retorno = temp_list
    
    return retorno

In [9]:
#listar todos os horarios(objetos) de um professor
def listar_horarios_professor(professor,objs_horarios):#listar todas os horarios(objetos) de um professor
    retorno = []
    dupla=professor.get_horarios()
    for x in range(len(objs_horarios)):
        horario=objs_horarios[x]
        for y in range(len(dupla)):
            dupla_codigo=dupla[y][1]
            dupla_dia=dupla[y][0]
            if(horario.get_codigo()==dupla_codigo and dupla_dia == horario.get_dia_semana()):
                retorno.append(horario)
                
    temp_list = []
    for i in retorno :
        if i not in temp_list:
            temp_list.append(i)

    retorno = temp_list
    
    return retorno

In [10]:
#listar todos os professores(objetos) de uma turma
def listar_professores_turma(turma,professores,objs):    
    retorno=[]
    for x in range(len(professores)):
            professor=professores[x]
            disciplinas_professor=professor.get_disciplinas()
            for y in range(len(disciplinas_professor)):
                indice=disciplinas_professor[y]
                disciplina=objs[indice]
                if (disciplina.get_numero_turma() == turma.get_codigo()):  
                    retorno.append(professor)
    return retorno

In [11]:
#listar todos os horarios de uma turma
def listar_horarios_turma(turma,professores,objs):    
    retorno=[]
    for x in range(len(professores)):
            professor=professores[x]
            disciplinas_professor=professor.get_disciplinas()
            for y in range(len(disciplinas_professor)):
                horarios=[]
                indice=disciplinas_professor[y]
                disciplina=objs[indice]
                if (disciplina.get_numero_turma() == turma.get_codigo()): 
                    horarios_disciplina=disciplina.get_horarios()
                    for z in range(len(horarios_disciplina)):
                        retorno.append(horarios_disciplina[z])
    temp_list = []

    for i in retorno :
        if i not in temp_list:
            temp_list.append(i)

    retorno = temp_list
    return retorno


In [12]:
#dado uma dupla dia e slot retorna o objeto correspondente a este horario
def get_obj_horario(lista,objs_horarios):
    retorno=[]
    for y in range(len(lista)):
        for x in range(len(objs_horarios)):
            horario=objs_horarios[x]
            if(lista[y]==[horario.get_dia_semana(),horario.get_codigo()]):
                retorno.append(horario)
    return retorno

In [13]:
#retorna lista de indices das disciplinas
def listar_disciplinas(objs):
    retorno=[]
    for x in range(len(objs)):
            disciplina=objs[x]
            retorno.append(disciplina.get_index())
    return retorno

In [14]:
#retorna lista de indices dos professores
def listar_professores(objs):
    retorno=[]
    for x in range(len(objs)):
            professor=objs[x]
            retorno.append(professor.get_index())
    return retorno

In [15]:
#retorna lista de indices dos horarios
def listar_horarios(objs):
    retorno=[]
    for x in range(len(objs)):
            horario=objs[x]
            retorno.append(horario.get_index())
    return retorno

In [16]:
#retorna lista de indices das turmas
def listar_turmas(objs):
    retorno=[]
    for x in range(len(objs)):
            turma=objs[x]
            retorno.append(turma.get_index())
    return retorno

In [17]:
#retorna lista de indices das disciplinas,horarios e professores de um objeto turma 
def gerar_lista_indices_turma(turma,objs,objs_professores,objs_horarios):
    
    disciplinas = listar_disciplinas(objs)
    professores = listar_professores(objs_professores)
    horarios = listar_horarios(objs_horarios)
    listaIndiceDisciplina= []
    listaIndiceProfessor=[]
    listaIndiceHorario=[]

    indiceTurma= turma.get_index() #turma
    disciplinas_da_turma=disciplinas_turma(turma,objs)
    professor_da_turma=listar_professores_turma(turma,objs_professores,objs)
    temp_horarios_da_turma=listar_horarios_turma(turma,objs_professores,objs)
    horarios_da_turma=get_obj_horario(temp_horarios_da_turma,objs_horarios)
    for y in range(len(disciplinas_da_turma)):
        disciplina=disciplinas_da_turma[y]
        indiceDisciplina=disciplina.get_index()
        listaIndiceDisciplina.append(indiceDisciplina)
        for z in range(len(professor_da_turma)):
            professor=professor_da_turma[z]
            indiceProfessor=professor.get_index()
            listaIndiceProfessor.append(indiceProfessor)
            for w in range(len(horarios_da_turma)):
                horario=horarios_da_turma[w]
                indiceHorario=horario.get_index()
                listaIndiceHorario.append(indiceHorario)
    
    temp_list = []
    for i in listaIndiceDisciplina :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceDisciplina = temp_list
    
    temp_list = []
    for i in listaIndiceProfessor :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceProfessor = temp_list
    
    temp_list = []
    for i in listaIndiceHorario :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceHorario = temp_list
    


    return (listaIndiceDisciplina,listaIndiceProfessor,listaIndiceHorario)

In [18]:
#retorna lista de indices das disciplinas,horarios e turmas de um objeto professor
def gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios):
    turmas = listar_turmas(objs_turmas)
    disciplinas = listar_disciplinas(objs)
    horarios = listar_horarios(objs_horarios)
    listaIndiceDisciplina= professor.get_disciplinas()
    listaIndiceTurma=[]
    listaIndiceHorario=[]
    
    indiceProfessor= professor.get_index() #professor
    
    turmas_professor=listar_turmas_professor(professor,objs_turmas,objs)
    horarios_do_professor=listar_horarios_professor(professor,objs_horarios)
    
    for z in range(len(turmas_professor)):
        turma=turmas_professor[z]
        indiceTurma=turma.get_index()
        listaIndiceTurma.append(indiceTurma)
    for w in range(len(horarios_do_professor)):
        horario=horarios_do_professor[w]
        indiceHorario=horario.get_index()
        listaIndiceHorario.append(indiceHorario)
    
    temp_list = []
    for i in listaIndiceDisciplina :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceDisciplina = temp_list    

    temp_list = []
    for i in listaIndiceTurma :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceTurma = temp_list
    
    temp_list = []
    for i in listaIndiceHorario :
        if i not in temp_list:
            temp_list.append(i)

    listaIndiceHorario = temp_list    
    
    return (listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario)

In [19]:
#criacao dos objetos disciplinas com os dados da tabela
lista_disciplinas = []
for i in df.index:
    lista_disciplinas.append([df['Código da disc. '][i],df['Código da área de ensino'][i],df['Qtd de aulas na semana'][i],df['Número da turma'][i],df['Início das aulas'][i],df['Fim das aulas'][i],df['Nome'][i]])

temp_list = []

for i in lista_disciplinas :
    if i not in temp_list:
        temp_list.append(i)

lista_disciplinas = temp_list

index=-1
objs = list()
for x in range(len(lista_disciplinas)):
    codigo = lista_disciplinas[x][0]
    area_ensino = lista_disciplinas[x][1]
    quantidade_aulas_semanais = lista_disciplinas[x][2]
    numero_turma = lista_disciplinas[x][3]
    inicio_aula = lista_disciplinas[x][4]
    fim_aula= lista_disciplinas[x][5]
    professor= lista_disciplinas[x][6]
    numero_maximo_diario= 3
    numero_maximo_sequencia= 3
    index=x
    objs.append(Disciplina(codigo,area_ensino,quantidade_aulas_semanais,numero_turma,inicio_aula,fim_aula,professor,numero_maximo_diario,numero_maximo_sequencia,index))

In [20]:
#criacao dos objetos turmas com os dados da tabela
lista_turmas = []
for i in df.index:
    lista_turmas.append([df['Número da turma'][i],df['Código da área de ensino'][i],df['Etapa (séries)'][i]])

temp_list = []

for i in lista_turmas :
    if i not in temp_list:
        temp_list.append(i)

lista_turmas = temp_list

index=-1
objs_turmas = list()
for x in range(len(lista_turmas)):
    codigo = lista_turmas[x][0]
    area_ensino = lista_turmas[x][1]
    serie = lista_turmas[x][2]
    index=x
    objs_turmas.append(Turma(codigo,area_ensino,serie,index))

In [21]:
#criacao dos objetos professores com os dados da tabela
lista_professor = []
for i in df.index:
    lista_professor.append([df['Identificador do prof. '][i],df['Nome'][i]])

temp_list = []

for i in lista_professor :
    if i not in temp_list:
        temp_list.append(i)

lista_professor = temp_list

index=-1
objs_professores = list()
for x in range(len(lista_professor)):
    lista_disciplinas=[]
    codigo = lista_professor[x][0]
    nome = lista_professor[x][1]
    for y in range(len(objs)):
        disciplina=objs[y]
        if(nome==disciplina.get_professor()):
            lista_disciplinas.append(disciplina.get_index())
    index=x
    objs_professores.append(Professor(codigo,nome,lista_disciplinas,index))

In [22]:
#criacao dos objetos grupos de disciplinas,nao utilizado nesta versao do modelo, pois nao foram adicionados restricoes relacionadas aos grupos
lista_0=["LÍNGUA PORTUGUESA E LITERATURA","LÍNGUA ESTRANGEIRA - INGLÊS","LÍNGUA PORTUGUESA","LETRAS - ENS. FUND.- ANOS INICIAIS","EDUCAÇÃO FÍSICA","ARTE"]
lista_1=["MATEMÁTICA"]
lista_2=["BIOLOGIA","QUÍMICA","FÍSICA","CIÊNCIAS"]
lista_3=["GEOGRAFIA","HISTÓRIA","SOCIOLOGIA","FILOSOFIA","ENSINO RELIGIOSO"]
grupo_0=[0,"Linguagens e suas Tecnologias",4,lista_0,0]
grupo_1=[1,"Matemática e suas Tecnologias",4,lista_1,1]
grupo_2=[2,"Ciências da Natureza e suas Tecnologias",4,lista_2,2]
grupo_3=[3,"Ciências Humanas e Sociais Aplicadas",4,lista_3,3]
lista_grupo = [grupo_0,grupo_1,grupo_2,grupo_3]

index=-1
objs_grupos = list()
for x in range(len(lista_grupo)):
    grupo_temp = lista_grupo[x]
    for y in range(len(grupo_temp)):
        codigo=grupo_temp[0]
        nome=grupo_temp[1]
        maximo=grupo_temp[2]
        disciplinas=grupo_temp[3]
        index=grupo_temp[4]
    objs_grupos.append(Grupo(codigo,nome,maximo,disciplinas,index))

In [23]:
#criacao dos objetos horarios com os dados da tabela
index=-1
objs_horarios = list()
for x in range (2,7):
    dia_semana=x
    for y in range (1,11):
        index=index+1
        codigo=y
        if(y>=1 and y<6):
            turno="Matutino"
            if(y==1):
                inicio="07:45"
            if(y==2):
                inicio="08:20"
            if(y==3):
                inicio="09:10"
            if(y==4):
                inicio="10:10"
            if(y==5):
                inicio="11:10"
            objs_horarios.append(Horario(codigo,dia_semana,inicio,turno,index))
        if(y>=6 and y<11):
            turno="Vespertino"
            if(y==6):
                inicio="13:30"
            if(y==7):
                inicio="14:15"
            if(y==8):
                inicio="15:25"
            if(y==9):
                inicio="16:00"
            if(y==10):
                inicio="16:50"
            objs_horarios.append(Horario(codigo,dia_semana,inicio,turno,index))
        if(y>=11):
            turno="Noturno"
            inicio="18:30"
            objs_horarios.append(Horario(codigo,dia_semana,inicio,turno,index))

In [24]:
#Inicializacao do modelo
model = gp.Model("Problema Horario")

Academic license - for non-commercial use only - expires 2022-07-28
Using license file C:\Users\Asus\gurobi.lic


In [25]:
#Variaveis de decisao

#conjunto de indices1:geracao de indices para as posibilidades de alocacao para cada turma os indices de disciplina,professor e horario
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)
variaveis= []

for x in range(len(turmas)):
    indiceTurma=0
    turma = objs_turmas[x]
    indiceTurma= turma.get_index() #turma
    disciplinas_da_turma=disciplinas_turma(turma,objs)
    professor_da_turma=listar_professores_turma(turma,objs_professores,objs)
    temp_horarios_da_turma=listar_horarios_turma(turma,objs_professores,objs)
    horarios_da_turma=get_obj_horario(temp_horarios_da_turma,objs_horarios)
    for y in range(len(disciplinas_da_turma)):
        disciplina=disciplinas_da_turma[y]
        indiceDisciplina=disciplina.get_index()
        for z in range(len(professor_da_turma)):
            professor=professor_da_turma[z]
            indiceProfessor=professor.get_index()
            for w in range(0,50):
                horario=objs_horarios[w]
                indiceHorario=horario.get_index()
                variaveis.append((indiceTurma,indiceDisciplina,indiceProfessor,indiceHorario))

In [26]:
#conjunto de indices2:geracao de indices para as posibilidades de alocacao para cada professor os indices de disciplina,turma e horario
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)

variaveis_rh2=[]
for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    for h in range(0,50):
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                variaveis_rh2.append((t,d,p,h))
for w in variaveis_rh2:
        variaveis.append(w)

#realizando interseccao de conjunto de indices1 e conjunto de indices2
set_variaveis=set(variaveis)
variaveis=list(set_variaveis)

In [27]:
#instanciando as variaveis relacionadas a alocao de horarios ao modelo
x = model.addVars(variaveis,vtype=gp.GRB.BINARY)

#conjunto de indices3:relacionados a janelas e dias de trabalho

#janelas
segunda_tarde=[]
terca_tarde=[]
quarta_tarde=[]
quinta_tarde=[]
sexta_tarde=[]

segunda_manha=[]
terca_manha=[]
quarta_manha=[]
quinta_manha=[]
sexta_manha=[]


#dias
segunda=[]
terca=[]
quarta=[]
quinta=[]
sexta=[]


for i in range(len(professores)):
    #dias
    segunda.append(i)
    terca.append(i)
    quarta.append(i)
    quinta.append(i)
    sexta.append(i)

    
    #janelas
    for h in range(5,8):
        segunda_tarde.append((i,h))
    for h in range(15,18):
        terca_tarde.append((i,h))
    for h in range(25,28):
        quarta_tarde.append((i,h))
    for h in range(35,38):
        quinta_tarde.append((i,h))
    for h in range(45,48):
        sexta_tarde.append((i,h))
        
    #janelas
    for h in range(0,3):
        segunda_manha.append((i,h))
    for h in range(10,13):
        terca_manha.append((i,h))
    for h in range(20,23):
        quarta_manha.append((i,h))
    for h in range(30,33):
        quinta_manha.append((i,h))
    for h in range(40,43):
        sexta_manha.append((i,h))
    
#instanciando as variaveis relacionadas a janelas e dias da semana em que o professor leciona
#janela
janela_segunda_tarde = model.addVars(segunda_tarde,vtype=gp.GRB.BINARY)
janela_terca_tarde = model.addVars(terca_tarde,vtype=gp.GRB.BINARY)
janela_quarta_tarde = model.addVars(quarta_tarde,vtype=gp.GRB.BINARY)
janela_quinta_tarde = model.addVars(quinta_tarde,vtype=gp.GRB.BINARY)
janela_sexta_tarde = model.addVars(sexta_tarde,vtype=gp.GRB.BINARY)

janela_segunda_manha = model.addVars(segunda_manha,vtype=gp.GRB.BINARY)
janela_terca_manha = model.addVars(terca_manha,vtype=gp.GRB.BINARY)
janela_quarta_manha = model.addVars(quarta_manha,vtype=gp.GRB.BINARY)
janela_quinta_manha = model.addVars(quinta_manha,vtype=gp.GRB.BINARY)
janela_sexta_manha = model.addVars(sexta_manha,vtype=gp.GRB.BINARY)


#dia
dia_segunda=model.addVars(segunda,vtype=gp.GRB.BINARY)
dia_terca=model.addVars(terca,vtype=gp.GRB.BINARY)
dia_quarta=model.addVars(quarta,vtype=gp.GRB.BINARY)
dia_quinta=model.addVars(quinta,vtype=gp.GRB.BINARY)
dia_sexta=model.addVars(sexta,vtype=gp.GRB.BINARY)



In [28]:
#conjunto de restricoes necessarias para identificar se um professor leciona em um dia na semana
for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()

    # Constants
    eps = 0.0001
    M   = 10 + eps  # smallest possible given bounds on x and y
    #seg
    # Model if x>y then b = 1, otherwise b = 0
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (0,10)) >= 0 + eps - M * (1 - dia_segunda[p]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_seg1")
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (0,10)) <= 0 + M * dia_segunda[p], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_seg2")
    
    #terca
    # Model if x>y then b = 1, otherwise b = 0
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (10,20)) >= 0 + eps - M * (1 - dia_terca[p]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_ter1")
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (10,20)) <= 0 + M * dia_terca[p], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_ter2")
    
    #quarta
    # Model if x>y then b = 1, otherwise b = 0
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (20,30)) >= 0 + eps - M * (1 - dia_quarta[p]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_qua1")
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (20,30)) <= 0 + M * dia_quarta[p], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_qua")


    #quinta
    # Model if x>y then b = 1, otherwise b = 0
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (30,40)) >= 0 + eps - M * (1 - dia_quinta[p]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_qui1")
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (30,40)) <= 0 + M * dia_quinta[p], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_qui2")

    #sexta
    # Model if x>y then b = 1, otherwise b = 0
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (40,50)) >= 0 + eps - M * (1 - dia_sexta[p]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_sex1")
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in range (40,50)) <= 0 + M * dia_sexta[p], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"bigM_sex2")
    

In [29]:
#conjunto de restricoes necessarias para janelas de um professor
for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    # Constants
    eps = 0.0001
    M   = 10 + eps  # smallest possible given bounds on x and y
    
    #tarde
    
    #segunda
    for h in range(5,8):
        # Model if x>y then b = 1, otherwise b = 0
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_segunda_tarde[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_seg1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_segunda_tarde[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_seg2")

    #terca
    for h in range(15,18):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_terca_tarde[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_ter1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_terca_tarde[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_ter2")

    #quarta
    for h in range(25,28):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_quarta_tarde[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qua1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_quarta_tarde[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qua2")

    #quinta    
    for h in range(35,38):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_quinta_tarde[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qui1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_quinta_tarde[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qui2")

        #sexta    
    for h in range(45,48):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_sexta_tarde[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_sex1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_sexta_tarde[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_sex2")
        
    
    #manha
    
    #segunda
    for h in range(0,3):
        # Model if x>y then b = 1, otherwise b = 0
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_segunda_manha[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_seg1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_segunda_manha[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_seg2")
    #terca
    for h in range(10,13):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_terca_manha[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_ter1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_terca_manha[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_ter2")

    #quarta
    for h in range(20,23):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_quarta_manha[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qua1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_quarta_manha[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qua2")

    #quinta    
    for h in range(30,33):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_quinta_manha[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qui1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_quinta_manha[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_qui2")

        #sexta    
    for h in range(40,43):
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) >= 0 + eps - M * (1 - janela_sexta_manha[p,h]), name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_sex1")
        model.addConstr((1 - ((1 - gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)) + gp.quicksum(x[t,d,p,h+1] for t in listaIndiceTurma for d in listaIndiceDisciplina) + (1 - gp.quicksum(x[t,d,p,h+2] for t in listaIndiceTurma for d in listaIndiceDisciplina)))) <= 0 + M * janela_sexta_manha[p,h], name="RestricaoObjetivo_ProfessorIndex_"+str(p)+"HorarioIndex_"+str(h)+"bigM_sex2")


In [30]:
#RH1:Para cada turma,cada horário em uma dia da semana é ocupado por no máximo uma disciplina nesta turma
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)

for z in range(len(turmas)):
    t=0
    turma = objs_turmas[z]
    listaIndiceDisciplina,listaIndiceProfessor,listaIndiceHorario=gerar_lista_indices_turma(turma,objs,objs_professores,objs_horarios)
    t=turma.get_index()
    for h in listaIndiceHorario:
        model.addConstr(gp.quicksum(x[t,d,p,h] for d in listaIndiceDisciplina for p in listaIndiceProfessor)<= 1,"RH1_TurmaIndex_"+str(t)+"__HorarioIndex_"+str(h))

In [31]:
#RH2:Cada professor pode lecionar no máximo uma disciplina em um mesmo horário que ele está disponível.
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)

for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    for h in listaIndiceHorario:
        model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina)<= 1,"RH2_ProfessorIndex_"+str(p)+"__HorarioIndex_"+str(h))

In [32]:
#RH3: Um professor não pode ultrapassar uma determinada quantidade máxima de aulas semanais.
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)


for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    aulas=professor.get_carga_horaria()
    model.addConstr(gp.quicksum(x[t,d,p,h] for t in listaIndiceTurma for d in listaIndiceDisciplina for h in listaIndiceHorario)==aulas,"RH3_ProfessorIndex_"+str(p))

In [33]:
#RH4: Uma turma t deve ter alocado exatamente o número de aulas previsto para a disciplina d lecionado pelo professor p, 
#conforme sua grade curricular.
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)

for z in range(len(turmas)):
    t=0
    turma = objs_turmas[z]
    listaIndiceDisciplina,listaIndiceProfessor,listaIndiceHorario=gerar_lista_indices_turma(turma,objs,objs_professores,objs_horarios)
    t=turma.get_index()
    for d in listaIndiceDisciplina:
        p=0
        disciplina=objs[d]
        professor_nome=disciplina.get_professor()
        for prof in listaIndiceProfessor:
            temp_professor=objs_professores[prof]
            temp_nome=temp_professor.get_nome()
            if(professor_nome==temp_nome):
                p=prof
                break
        aulas=disciplina.get_quantidade_aulas_semanais()
        model.addConstr(gp.quicksum(x[t,d,p,h] for h in listaIndiceHorario)== aulas,"RH4_TurmaIndex_"+str(t)+"__DisciplinaIndex_"+str(d)+"__ProfessorIndex_"+str(p))


In [34]:
#RH5:Cada disciplina de uma turma  ́e dada no maximo 1 vez por dia da semana
turmas = listar_turmas(objs_turmas)
disciplinas = listar_disciplinas(objs)
professores = listar_professores(objs_professores)
horarios = listar_horarios(objs_horarios)
diadasemana=[(0,1,2,3,4,5,6,7,8,9),(10,11,12,13,14,15,16,17,18,19),(20,21,22,23,24,25,26,27,28,29),(30,31,32,33,34,35,36,37,38,39),(40,41,42,43,44,45,46,47,48,49)]

for z in range(len(turmas)):
    t=0
    turma = objs_turmas[z]
    listaIndiceDisciplina,listaIndiceProfessor,listaIndiceHorario=gerar_lista_indices_turma(turma,objs,objs_professores,objs_horarios)
    t=turma.get_index()
    for dia in range(len(diadasemana)):
        indices_do_dia=diadasemana[dia]
        myList=indices_do_dia
        for indice in indices_do_dia:
            if(indice not in listaIndiceHorario):
                valueToBeRemoved = indice
                myList = [value for value in myList if value != valueToBeRemoved]
        for d in listaIndiceDisciplina:
            model.addConstr(gp.quicksum(x[t,d,p,h] for p in listaIndiceProfessor for h in myList)<= 1,"RH6_TurmaIndex_"+str(t)+"__DisciplinaIndex_"+str(d)+"__DiaDaSemanaIndex_"+str(dia))

In [35]:

#funcao objetivo combinando
model.setObjective(gp.quicksum(janela_segunda_tarde[p,h] for p in range(len(professores)) for h in range(5,8))+gp.quicksum(janela_terca_tarde[p,h] for p in range(len(professores)) for h in range(15,18))+gp.quicksum(janela_quarta_tarde[p,h] for p in range(len(professores)) for h in range(25,28))+gp.quicksum(janela_quinta_tarde[p,h] for p in range(len(professores)) for h in range(35,38))+gp.quicksum(janela_sexta_tarde[p,h] for p in range(len(professores)) for h in range(45,48))+gp.quicksum(janela_segunda_manha[p,h] for p in range(len(professores)) for h in range(0,3))+gp.quicksum(janela_terca_manha[p,h] for p in range(len(professores)) for h in range(10,13))+gp.quicksum(janela_quarta_manha[p,h] for p in range(len(professores)) for h in range(20,23))+gp.quicksum(janela_quinta_manha[p,h] for p in range(len(professores)) for h in range(30,33))+gp.quicksum(janela_sexta_manha[p,h] for p in range(len(professores)) for h in range(40,43))+6*gp.quicksum(dia_segunda[p] + dia_terca[p] + dia_quarta[p] + dia_quinta[p] + dia_sexta[p] for p in range(len(professores))),sense = gp.GRB.MINIMIZE)

In [36]:
model.Params.WorkLimit = 10*60
model.optimize()

No parameters matching 'WorkLimit' found
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 3432 rows, 110470 columns and 404760 nonzeros
Model fingerprint: 0x5a934abc
Variable types: 0 continuous, 110470 integer (110470 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 367 rows and 106575 columns
Presolve time: 0.46s
Presolved: 3065 rows, 3895 columns, 30175 nonzeros
Variable types: 0 continuous, 3895 integer (3745 binary)

Root relaxation: objective 2.940000e+02, 5260 iterations, 0.89 seconds
Total elapsed time = 5.24s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  294.00000    0   68          -  294.00000      -     -    5s
H    0   

In [37]:
#printar grade horaria professor
for z in range(len(professores)):
    p=0
    atual=0
    anterior=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    contador=0
    print(" ")
    print("-----------------------------------------------------------------------------------")
    print("| Prof: ",professor.get_nome(),"|")
    for h in listaIndiceHorario:
        horario=objs_horarios[h]
        atual=horario.get_dia_semana()
        for t in listaIndiceTurma:
            turma=objs_turmas[t]
            for d in listaIndiceDisciplina:
                disciplina=objs[d]
                if (round(x[t,d,p,h].X)==1):
                    contador=contador+1
                    if(atual!=anterior):
                        print("-----------------------------------------------------------------------------------")
                    anterior=horario.get_dia_semana()
                    print("|%15s %1d | %6s %2d | %12s %16s | %6s %4s |" %("Dia da semana: ",horario.get_dia_semana(),"Slot: ",horario.get_codigo(),"Disciplina: ",disciplina.nome(),"Turma: ",round(turma.get_codigo())))
                    #print("| Dia da Semana: ",horario.get_dia_semana(),"\N{DEGREE SIGN}","| Slot: ",horario.get_codigo(),"| Disciplina: ",disciplina.nome(),"| Turma: ",round(turma.get_codigo()),"| Aula: ",round(x[t,d,p,h].X),"|")
    print(" ")
    print("| Carga horária: ",contador,"|")
    print(" ")

 
-----------------------------------------------------------------------------------
| Prof:  P1 |
-----------------------------------------------------------------------------------
|Dia da semana:  3 | Slot:   2 | Disciplina:        SOCIOLOGIA | Turma:   100 |
|Dia da semana:  3 | Slot:   3 | Disciplina:        SOCIOLOGIA | Turma:   201 |
|Dia da semana:  3 | Slot:   4 | Disciplina:        SOCIOLOGIA | Turma:   101 |
|Dia da semana:  3 | Slot:   5 | Disciplina:        SOCIOLOGIA | Turma:   200 |
|Dia da semana:  3 | Slot:   6 | Disciplina:        SOCIOLOGIA | Turma:   302 |
|Dia da semana:  3 | Slot:   7 | Disciplina:        SOCIOLOGIA | Turma:   103 |
|Dia da semana:  3 | Slot:   8 | Disciplina:        SOCIOLOGIA | Turma:   102 |
|Dia da semana:  3 | Slot:   9 | Disciplina:        SOCIOLOGIA | Turma:   203 |
|Dia da semana:  3 | Slot:  10 | Disciplina:        SOCIOLOGIA | Turma:   202 |
-----------------------------------------------------------------------------------
|Dia da sema

|Dia da semana:  6 | Slot:   4 | Disciplina:            FÍSICA | Turma:   100 |
|Dia da semana:  6 | Slot:   5 | Disciplina:            FÍSICA | Turma:   200 |
|Dia da semana:  6 | Slot:   6 | Disciplina:            FÍSICA | Turma:   302 |
|Dia da semana:  6 | Slot:   7 | Disciplina:            FÍSICA | Turma:   103 |
|Dia da semana:  6 | Slot:   8 | Disciplina:            FÍSICA | Turma:   202 |
|Dia da semana:  6 | Slot:   9 | Disciplina:            FÍSICA | Turma:   102 |
|Dia da semana:  6 | Slot:  10 | Disciplina:            FÍSICA | Turma:   203 |
 
| Carga horária:  20 |
 
 
-----------------------------------------------------------------------------------
| Prof:  P9 |
-----------------------------------------------------------------------------------
|Dia da semana:  2 | Slot:   1 | Disciplina:         GEOGRAFIA | Turma:   200 |
|Dia da semana:  2 | Slot:   2 | Disciplina:         GEOGRAFIA | Turma:   201 |
|Dia da semana:  2 | Slot:   3 | Disciplina:         GEOGRAFIA | Turm

In [38]:
#printar grade horaria turma
for z in range(len(turmas)):
    atual=0
    anterior=0
    t=0
    turma = objs_turmas[z]
    listaIndiceDisciplina,listaIndiceProfessor,listaIndiceHorario=gerar_lista_indices_turma(turma,objs,objs_professores,objs_horarios)
    t=turma.get_index()
    contador=0
    print(" ")
    print("-----------------------------------------------------------------------------------")
    print("| Turma: ",round(turma.get_codigo()),"|")
    for h in listaIndiceHorario:
        horario=objs_horarios[h]
        atual=horario.get_dia_semana()
        if(atual!=anterior):
            print("-----------------------------------------------------------------------------------")
        for d in listaIndiceDisciplina:
            disciplina=objs[d]
            for p in listaIndiceProfessor:
                professor = objs_professores[p]
                if (round(x[t,d,p,h].X)==1):
                    contador=contador+1
                    anterior=horario.get_dia_semana()
                    print("|%15s %1d | %6s %2d | %12s %16s | %12s %3s |" %("Dia da semana: ",horario.get_dia_semana(),"Slot: ",horario.get_codigo(),"Disciplina: ",disciplina.nome(),"Professor: ",professor.get_nome()))
                    #print("| Dia da Semana: ",horario.get_dia_semana(),"\N{DEGREE SIGN}","| Slot: ",horario.get_codigo(),"| Disciplina: ",disciplina.nome(),"| Professor: ",professor.get_nome(),"| Aula: ",round(x[t,d,p,h].X),"|")
    print("-----------------------------------------------------------------------------------")
    print(" ")
    print("|Carga horária: ",contador,"|")
    print(" ")

 
-----------------------------------------------------------------------------------
| Turma:  300 |
-----------------------------------------------------------------------------------
|Dia da semana:  2 | Slot:   1 | Disciplina:           QUÍMICA |  Professor:  P11 |
|Dia da semana:  2 | Slot:   2 | Disciplina:          HISTÓRIA |  Professor:   P7 |
|Dia da semana:  2 | Slot:   3 | Disciplina:        MATEMÁTICA |  Professor:   P3 |
|Dia da semana:  2 | Slot:   4 | Disciplina:        LITERATURA |  Professor:   P2 |
|Dia da semana:  2 | Slot:   5 | Disciplina:         GEOGRAFIA |  Professor:   P9 |
-----------------------------------------------------------------------------------
|Dia da semana:  3 | Slot:   1 | Disciplina:          BIOLOGIA |  Professor:   P6 |
|Dia da semana:  3 | Slot:   2 | Disciplina:        MATEMÁTICA |  Professor:   P3 |
|Dia da semana:  3 | Slot:   3 | Disciplina:         FILOSOFIA |  Professor:  P10 |
|Dia da semana:  3 | Slot:   4 | Disciplina:            IN

|Dia da semana:  3 | Slot:   8 | Disciplina:   EDUCAÇÃO FÍSICA |  Professor:  P17 |
|Dia da semana:  3 | Slot:   9 | Disciplina:        SOCIOLOGIA |  Professor:   P1 |
|Dia da semana:  3 | Slot:  10 | Disciplina:         FILOSOFIA |  Professor:  P10 |
-----------------------------------------------------------------------------------
|Dia da semana:  4 | Slot:   6 | Disciplina:        MATEMÁTICA |  Professor:  P14 |
|Dia da semana:  4 | Slot:   7 | Disciplina:           QUÍMICA |  Professor:  P11 |
|Dia da semana:  4 | Slot:   8 | Disciplina:              ARTE |  Professor:   P4 |
|Dia da semana:  4 | Slot:   9 | Disciplina:            FÍSICA |  Professor:   P8 |
|Dia da semana:  4 | Slot:  10 | Disciplina:         FILOSOFIA |  Professor:  P10 |
-----------------------------------------------------------------------------------
|Dia da semana:  5 | Slot:   6 | Disciplina:        LITERATURA |  Professor:   P2 |
|Dia da semana:  5 | Slot:   7 | Disciplina:   EDUCAÇÃO FÍSICA |  Professor:

In [39]:
#printar slots
print(" ")
print("-----------Manha---------------")
print(" ")
print("-----------Segunda-------------")
for h in range (0,5):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Terca-------------")
for h in range (10,15):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Quarta-------------")
for h in range (20,25):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Quinta-------------")
for h in range (30,35):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Sexta-------------")
for h in range (40,45):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Tarde---------------")
print(" ")
print("-----------Segunda-------------")
for h in range (5,10):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Terca-------------")
for h in range (15,20):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Quarta-------------")
for h in range (25,30):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Quinta-------------")
for h in range (35,40):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")
print(" ")
print("-----------Sexta-------------")
for h in range (45,50):
    horariox=objs_horarios[h]
    print("| SLOT:",horariox.get_codigo(),"| Inicio: ",horariox.get_inicio(),"|")

 
-----------Manha---------------
 
-----------Segunda-------------
| SLOT: 1 | Inicio:  07:45 |
| SLOT: 2 | Inicio:  08:20 |
| SLOT: 3 | Inicio:  09:10 |
| SLOT: 4 | Inicio:  10:10 |
| SLOT: 5 | Inicio:  11:10 |
 
-----------Terca-------------
| SLOT: 1 | Inicio:  07:45 |
| SLOT: 2 | Inicio:  08:20 |
| SLOT: 3 | Inicio:  09:10 |
| SLOT: 4 | Inicio:  10:10 |
| SLOT: 5 | Inicio:  11:10 |
 
-----------Quarta-------------
| SLOT: 1 | Inicio:  07:45 |
| SLOT: 2 | Inicio:  08:20 |
| SLOT: 3 | Inicio:  09:10 |
| SLOT: 4 | Inicio:  10:10 |
| SLOT: 5 | Inicio:  11:10 |
 
-----------Quinta-------------
| SLOT: 1 | Inicio:  07:45 |
| SLOT: 2 | Inicio:  08:20 |
| SLOT: 3 | Inicio:  09:10 |
| SLOT: 4 | Inicio:  10:10 |
| SLOT: 5 | Inicio:  11:10 |
 
-----------Sexta-------------
| SLOT: 1 | Inicio:  07:45 |
| SLOT: 2 | Inicio:  08:20 |
| SLOT: 3 | Inicio:  09:10 |
| SLOT: 4 | Inicio:  10:10 |
| SLOT: 5 | Inicio:  11:10 |
 
-----------Tarde---------------
 
-----------Segunda-------------
| SLOT: 6

In [40]:
print("----------Analise de resultados Tarde ----------")
print("")
total=0
janela_semanal_total=0
print("|--------------------------Siglas------------------------------|")
print("|h1 corresponde a um slot de horario                           |")
print("|h2 corresponde ao slot sequente de h1                         |")
print("|h3 corresponde ao slot sequente de h2                         |")
print("|j corresponde se existe janela                                |")
print("|p corresponde a penalidade atribuido a sequencia de h1,h2,h3  |")
print("|--------------------------------------------------------------|")
print("|penalidade aplicada nesta execucao,se foi adicionada na funcao objetivo|")
print("")
for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    janela_segunda=0
    janela_terca=0
    janela_quarta=0
    janela_quinta=0
    janela_sexta=0
    janela_semanal=0
    janela_semanal_real=0
    print("----------------------------------------------------")
    print("Professor:",professor.get_nome())

    print("-----------Segunda-------------")
    print("h1","h2","h3","j","  p")
    for h in range (5,8):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_segunda=janela_segunda+(a*(1-b)*c)
        j=a*(1-b)*c
        pen= a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)
        

    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,5].X)
            slot2=slot2+round(x[t,d,p,6].X)
            slot3=slot3+round(x[t,d,p,7].X)
            slot4=slot4+round(x[t,d,p,8].X)
            slot5=slot5+round(x[t,d,p,9].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_segunda=0
    contador_janelas_segunda=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_segunda=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_segunda=0
                        break
                    if(lista[e]==0):
                        contador_janelas_segunda=contador_janelas_segunda+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_segunda>0):
                    pular=contador_janelas_segunda+1
                    janelas_existentes_segunda=janelas_existentes_segunda+contador_janelas_segunda
    print(" ")
    print("janelas analisando h1,h2,h3 na segunda =",janela_segunda)
    print(" ")
    print("janelas existentes na segunda =",janelas_existentes_segunda)
    
    print("-----------Terca-------------")
    print("h1","h2","h3","j","  p")
    for h in range (15,18):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_terca=janela_terca+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)
        
        
        
    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,15].X)
            slot2=slot2+round(x[t,d,p,16].X)
            slot3=slot3+round(x[t,d,p,17].X)
            slot4=slot4+round(x[t,d,p,18].X)
            slot5=slot5+round(x[t,d,p,19].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_terca=0
    contador_janelas_terca=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_terca=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_terca=0
                        break
                    if(lista[e]==0):
                        contador_janelas_terca=contador_janelas_terca+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_terca>0):
                    pular=contador_janelas_terca+1
                    janelas_existentes_terca=janelas_existentes_terca+contador_janelas_terca
    print(" ")
    print("janelas analisando h1,h2,h3 na terca =",janela_terca)
    print(" ")
    print("janelas existentes na terca=",janelas_existentes_terca)
    
    print("-----------Quarta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (25,28):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_quarta=janela_quarta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)

        
        
        
    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,25].X)
            slot2=slot2+round(x[t,d,p,26].X)
            slot3=slot3+round(x[t,d,p,27].X)
            slot4=slot4+round(x[t,d,p,28].X)
            slot5=slot5+round(x[t,d,p,29].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_quarta=0
    contador_janelas_quarta=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_quarta=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_quarta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_quarta=contador_janelas_quarta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_quarta>0):
                    pular=contador_janelas_quarta+1
                    janelas_existentes_quarta=janelas_existentes_quarta+contador_janelas_quarta
    print(" ")
    print("janelas analisando h1,h2,h3 na quarta =",janela_quarta)
    print(" ")
    print("janelas existentes na quarta=",janelas_existentes_quarta)
    
    print("-----------Quinta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (35,38):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_quinta=janela_quinta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)

    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,35].X)
            slot2=slot2+round(x[t,d,p,36].X)
            slot3=slot3+round(x[t,d,p,37].X)
            slot4=slot4+round(x[t,d,p,38].X)
            slot5=slot5+round(x[t,d,p,39].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_quinta=0
    contador_janelas_quinta=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_quinta=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_quinta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_quinta=contador_janelas_quinta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_quinta>0):
                    pular=contador_janelas_quinta+1
                    janelas_existentes_quinta=janelas_existentes_quinta+contador_janelas_quinta
    print(" ")
    print("janelas analisando h1,h2,h3 na quinta =",janela_quinta)
    print(" ")
    print("janelas existentes na quinta=",janelas_existentes_quinta)

    print("-----------Sexta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (45,48):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_sexta=janela_sexta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)


    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,45].X)
            slot2=slot2+round(x[t,d,p,46].X)
            slot3=slot3+round(x[t,d,p,47].X)
            slot4=slot4+round(x[t,d,p,48].X)
            slot5=slot5+round(x[t,d,p,49].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_sexta=0
    contador_janelas_sexta=0
    pular=0
    for b in range(0,5):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_sexta=0
                for e in range(b+1,5):
                    if(e==5 and lista[e]==0):
                        contador_janelas_sexta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_sexta=contador_janelas_sexta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_sexta>0):
                    pular=contador_janelas_quinta+1
                    janelas_existentes_sexta=janelas_existentes_sexta+contador_janelas_sexta
    print(" ")
    print("janelas analisando h1,h2,h3 na sexta =",janela_sexta)
    print(" ")
    print("janelas existentes na sexta=",janelas_existentes_sexta)
    
    janela_semanal=janela_segunda+janela_terca+janela_quarta+janela_quinta+janela_sexta
    janela_semanal_real=janelas_existentes_segunda+janelas_existentes_terca+janelas_existentes_quarta+janelas_existentes_quinta+janelas_existentes_sexta
    
    
    print("-----------Semana-------------")
    print("|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA =",janela_semanal,"|")
    print("|TOTAL DE JANELAS EXISTENTES NA SEMANA =",janela_semanal_real,"|")
    total=janela_semanal+total
    janela_semanal_total=janela_semanal_total+janela_semanal_real
print(" ")
print("------------FIM--------------")
print("|TOTAL DE JANELAS ANALISANDO H1,H2,H3 TARDE=",total,"|")
print("|TOTAL DE JANELAS EXISTENTES NESSA SOLUCAO TURNO DA TARDE=",janela_semanal_total,"|")
diasAlocados=gp.quicksum(dia_segunda[p].X + dia_terca[p].X + dia_quarta[p].X + dia_quinta[p].X + dia_sexta[p].X for p in range(len(professores)))
print("|Dias alocados manha/tarde: ",diasAlocados,"|")
    

----------Analise de resultados Tarde ----------

|--------------------------Siglas------------------------------|
|h1 corresponde a um slot de horario                           |
|h2 corresponde ao slot sequente de h1                         |
|h3 corresponde ao slot sequente de h2                         |
|j corresponde se existe janela                                |
|p corresponde a penalidade atribuido a sequencia de h1,h2,h3  |
|--------------------------------------------------------------|
|penalidade aplicada nesta execucao,se foi adicionada na funcao objetivo|

----------------------------------------------------
Professor: P1
-----------Segunda-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Qua

0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quinta = 0
 
janelas existentes na quinta= 0
-----------Sexta-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na sexta = 0
 
janelas existentes na sexta= 0
-----------Semana-------------
|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA = 0 |
|TOTAL DE JANELAS EXISTENTES NA SEMANA = 0 |
----------------------------------------------------
Professor: P9
-----------Segunda-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Quarta-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quarta = 0
 
janelas existentes na quarta= 0
------

0  1  0  0   0
 
janelas analisando h1,h2,h3 na sexta = 0
 
janelas existentes na sexta= 3
-----------Semana-------------
|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA = 0 |
|TOTAL DE JANELAS EXISTENTES NA SEMANA = 3 |
----------------------------------------------------
Professor: P17
-----------Segunda-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  0  0   0
1  0  0  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Quarta-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quarta = 0
 
janelas existentes na quarta= 0
-----------Quinta-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  0  0   0
1  0  0  0   0
 
janelas analisando h1,h2,h3 na quinta = 0
 
janelas existentes na quinta= 0
-----------Sexta-------------
h1 h

In [41]:
print("----------Analise de resultados MANHA ----------")
print("")
total=0
janela_semanal_total=0
print("|--------------------------Siglas------------------------------|")
print("|h1 corresponde a um slot de horario                           |")
print("|h2 corresponde ao slot sequente de h1                         |")
print("|h3 corresponde ao slot sequente de h2                         |")
print("|j corresponde se existe janela                                |")
print("|p corresponde a penalidade atribuido a sequencia de h1,h2,h3  |")
print("|--------------------------------------------------------------|")
print("|penalidade aplicada nesta execucao,se foi adicionada na funcao objetivo|")
print("")
for z in range(len(professores)):
    p=0
    professor = objs_professores[z]
    listaIndiceDisciplina,listaIndiceTurma,listaIndiceHorario=gerar_lista_indices_professor(professor,objs,objs_turmas,objs_horarios)
    p=professor.get_index()
    janela_segunda=0
    janela_terca=0
    janela_quarta=0
    janela_quinta=0
    janela_sexta=0
    janela_semanal=0
    janela_semanal_real=0
    print("----------------------------------------------------")
    print("Professor:",professor.get_nome())

    print("-----------Segunda-------------")
    print("h1","h2","h3","j","  p")
    for h in range (0,3):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_segunda=janela_segunda+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)
        

    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,0].X)
            slot2=slot2+round(x[t,d,p,1].X)
            slot3=slot3+round(x[t,d,p,2].X)
            slot4=slot4+round(x[t,d,p,3].X)
            slot5=slot5+round(x[t,d,p,4].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_segunda=0
    contador_janelas_segunda=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_segunda=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_segunda=0
                        break
                    if(lista[e]==0):
                        contador_janelas_segunda=contador_janelas_segunda+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_segunda>0):
                    pular=contador_janelas_segunda+1
                    janelas_existentes_segunda=janelas_existentes_segunda+contador_janelas_segunda
    print(" ")
    print("janelas analisando h1,h2,h3 na segunda =",janela_segunda)
    print(" ")
    print("janelas existentes na segunda =",janelas_existentes_segunda)
    
    print("-----------Terca-------------")
    print("h1","h2","h3","j","  p")
    for h in range (10,13):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_terca=janela_terca+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)
        
        
        
    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,10].X)
            slot2=slot2+round(x[t,d,p,11].X)
            slot3=slot3+round(x[t,d,p,12].X)
            slot4=slot4+round(x[t,d,p,13].X)
            slot5=slot5+round(x[t,d,p,14].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_terca=0
    contador_janelas_terca=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_terca=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_terca=0
                        break
                    if(lista[e]==0):
                        contador_janelas_terca=contador_janelas_terca+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_terca>0):
                    pular=contador_janelas_terca+1
                    janelas_existentes_terca=janelas_existentes_terca+contador_janelas_terca
    print(" ")
    print("janelas analisando h1,h2,h3 na terca =",janela_terca)
    print(" ")
    print("janelas existentes na terca=",janelas_existentes_terca)
    
    print("-----------Quarta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (20,23):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_quarta=janela_quarta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)

        
        
        
    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,20].X)
            slot2=slot2+round(x[t,d,p,21].X)
            slot3=slot3+round(x[t,d,p,22].X)
            slot4=slot4+round(x[t,d,p,23].X)
            slot5=slot5+round(x[t,d,p,24].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_quarta=0
    contador_janelas_quarta=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_quarta=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_quarta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_quarta=contador_janelas_quarta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_quarta>0):
                    pular=contador_janelas_quarta+1
                    janelas_existentes_quarta=janelas_existentes_quarta+contador_janelas_quarta
    print(" ")
    print("janelas analisando h1,h2,h3 na quarta =",janela_quarta)
    print(" ")
    print("janelas existentes na quarta=",janelas_existentes_quarta)
    
    print("-----------Quinta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (30,33):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_quinta=janela_quinta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)

    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,30].X)
            slot2=slot2+round(x[t,d,p,31].X)
            slot3=slot3+round(x[t,d,p,32].X)
            slot4=slot4+round(x[t,d,p,33].X)
            slot5=slot5+round(x[t,d,p,34].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_quinta=0
    contador_janelas_quinta=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_quinta=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_quinta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_quinta=contador_janelas_quinta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_quinta>0):
                    pular=contador_janelas_quinta+1
                    janelas_existentes_quinta=janelas_existentes_quinta+contador_janelas_quinta
    print(" ")
    print("janelas analisando h1,h2,h3 na quinta =",janela_quinta)
    print(" ")
    print("janelas existentes na quinta=",janelas_existentes_quinta)

    print("-----------Sexta-------------")
    print("h1","h2","h3","j","  p")
    for h in range (40,43):
        a=0
        b=0
        c=0
        j=0
        pen=0
        for t in listaIndiceTurma:
            for d in listaIndiceDisciplina:
                a=a+round(x[t,d,p,h].X)
                b=b+round(x[t,d,p,h+1].X)
                c=c+round(x[t,d,p,h+2].X)
        janela_sexta=janela_sexta+(a*(1-b)*c)
        j=a*(1-b)*c
        pen=a*(1-b)*c
        print(a,"",b,"",c,"",j," ",pen)


    slot1=0
    slot2=0
    slot3=0
    slot4=0
    slot5=0
    lista=[slot1,slot2,slot3,slot4,slot5]
    for t in listaIndiceTurma:
        for d in listaIndiceDisciplina:
            slot1=slot1+round(x[t,d,p,40].X)
            slot2=slot2+round(x[t,d,p,41].X)
            slot3=slot3+round(x[t,d,p,42].X)
            slot4=slot4+round(x[t,d,p,43].X)
            slot5=slot5+round(x[t,d,p,44].X)
    lista=[slot1,slot2,slot3,slot4,slot5]
    janelas_existentes_sexta=0
    contador_janelas_sexta=0
    pular=0
    for b in range(0,4):
        if(pular>0):
            pular=pular-1
        if(pular==0):
            if(lista[b]==1):
                contador_janelas_sexta=0
                for e in range(b+1,5):
                    if(e==4 and lista[e]==0):
                        contador_janelas_sexta=0
                        break
                    if(lista[e]==0):
                        contador_janelas_sexta=contador_janelas_sexta+1
                    if(e!=b and lista[e]==1):
                        break
                if(contador_janelas_sexta>0):
                    pular=contador_janelas_quinta+1
                    janelas_existentes_sexta=janelas_existentes_sexta+contador_janelas_sexta
    print(" ")
    print("janelas analisando h1,h2,h3 na sexta =",janela_sexta)
    print(" ")
    print("janelas existentes na sexta=",janelas_existentes_sexta)
    
    janela_semanal=janela_segunda+janela_terca+janela_quarta+janela_quinta+janela_sexta
    janela_semanal_real=janelas_existentes_segunda+janelas_existentes_terca+janelas_existentes_quarta+janelas_existentes_quinta+janelas_existentes_sexta
    
    
    print("-----------Semana-------------")
    print("|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA =",janela_semanal,"|")
    print("|TOTAL DE JANELAS EXISTENTES NA SEMANA =",janela_semanal_real,"|")
    total=janela_semanal+total
    janela_semanal_total=janela_semanal_total+janela_semanal_real
print(" ")
print("------------FIM--------------")
print("|TOTAL DE JANELAS ANALISANDO H1,H2,H3 =",total,"|")
print("|TOTAL DE JANELAS EXISTENTES NESSA SOLUCAO MANHA =",janela_semanal_total,"|")
diasAlocados=gp.quicksum(dia_segunda[p].X + dia_terca[p].X + dia_quarta[p].X + dia_quinta[p].X + dia_sexta[p].X for p in range(len(professores)))
print("|Dias alocados manha/tarde: ",diasAlocados,"|")
    

----------Analise de resultados MANHA ----------

|--------------------------Siglas------------------------------|
|h1 corresponde a um slot de horario                           |
|h2 corresponde ao slot sequente de h1                         |
|h3 corresponde ao slot sequente de h2                         |
|j corresponde se existe janela                                |
|p corresponde a penalidade atribuido a sequencia de h1,h2,h3  |
|--------------------------------------------------------------|
|penalidade aplicada nesta execucao,se foi adicionada na funcao objetivo|

----------------------------------------------------
Professor: P1
-----------Segunda-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
0  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Qua

0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quinta = 0
 
janelas existentes na quinta= 0
-----------Sexta-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na sexta = 0
 
janelas existentes na sexta= 0
-----------Semana-------------
|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA = 0 |
|TOTAL DE JANELAS EXISTENTES NA SEMANA = 0 |
----------------------------------------------------
Professor: P9
-----------Segunda-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  1  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Quarta-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quarta = 0
 
janelas existentes na quarta= 0
------

1  0  0  0   0
 
janelas analisando h1,h2,h3 na sexta = 0
 
janelas existentes na sexta= 0
-----------Semana-------------
|TOTAL DE JANELAS ANALISANDO H1,H2,H3 NA SEMANA = 0 |
|TOTAL DE JANELAS EXISTENTES NA SEMANA = 0 |
----------------------------------------------------
Professor: P17
-----------Segunda-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na segunda = 0
 
janelas existentes na segunda = 0
-----------Terca-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  0  0   0
 
janelas analisando h1,h2,h3 na terca = 0
 
janelas existentes na terca= 0
-----------Quarta-------------
h1 h2 h3 j   p
0  0  0  0   0
0  0  0  0   0
0  0  0  0   0
 
janelas analisando h1,h2,h3 na quarta = 0
 
janelas existentes na quarta= 0
-----------Quinta-------------
h1 h2 h3 j   p
1  1  1  0   0
1  1  1  0   0
1  1  0  0   0
 
janelas analisando h1,h2,h3 na quinta = 0
 
janelas existentes na quinta= 0
-----------Sexta-------------
h1 h

In [42]:
model.write("teste_full_V3_GRAD_5.lp")
model.dispose()